In [ ]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import csv
import time
import random

# 输入FASTA文件
fasta_file = "sampled_100_proteins.fasta"
output_csv = "homology_pairs.csv"

# 参数
max_homologs_per_query = 5  # 每条取1-5条同源
e_value_thresh = 1e-10      # e-value阈值，越小越严格
identity_thresh = 30.0      # 最低身份度百分比（>30%通常视为同源）

# 打开CSV写入
with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["sentence1", "sentence2", "label"])

    # 读取所有序列
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    print(f"总共 {len(sequences)} 条序列，开始处理...")

    for idx, record in enumerate(sequences, 1):
        query_seq = str(record.seq)
        query_id = record.id
        print(f"[{idx}/{len(sequences)}] 处理序列: {query_id} (长度: {len(query_seq)})")

        try:
            # 运行在线BLAST（对swissprot数据库）
            print("  正在运行BLAST...")
            result_handle = NCBIWWW.qblast(
                program="blastp", 
                database="swissprot", 
                sequence=query_seq,
                expect=e_value_thresh,   # 直接设置expect过滤
                hitlist_size=20          # 先取多一点，后面再过滤identity
            )

            # 解析结果
            blast_records = NCBIXML.parse(result_handle)
            blast_record = next(blast_records, None)  # 只有一个record

            homolog_count = 0
            if blast_record:
                for alignment in blast_record.alignments:
                    hit_id = alignment.hit_id   # 如 sp|P12345|XXX
                    hit_seq = ""  # 我们稍后从HSP取对齐部分，但这里用完整hit序列（NCBI返回的是对齐，但我们取第一个HSP的hit序列）
                    
                    for hsp in alignment.hsps:
                        hit_seq = hsp.sbjct  # 对齐的hit部分（保守）
                        identity = (hsp.identities / hsp.align_length) * 100

                        if identity >= identity_thresh:
                            # 排除自身（如果query本身在数据库中）
                            if hit_id.split("|")[1] == query_id:  # UniProt ID匹配
                                continue

                            # 写入一对
                            writer.writerow([query_seq, hsp.sbjct, 1])  # 用对齐序列作为sentence2，更公平（长度一致）
                            # 如果想用完整hit序列：需额外请求，但复杂，这里用对齐部分
                            homolog_count += 1
                            print(f"    找到同源: {hit_id} (identity: {identity:.1f}%, e-value: {hsp.expect:.0e})")

                            if homolog_count >= max_homologs_per_query:
                                break
                    if homolog_count >= max_homologs_per_query:
                        break

            if homolog_count == 0:
                print("  无合格同源序列。")

        except Exception as e:
            print(f"  BLAST错误: {e}，跳过此序列。")

        # 礼貌等待，避免NCBI限速
        time.sleep(3 + random.uniform(0, 3))  # 3-6秒随机间隔

print(f"完成！正样本对已保存到 {output_csv}")

总共 100 条序列，开始处理...
[1/100] 处理序列: sp|Q3C1K4|CLPP_NICSY (长度: 196)
  正在运行BLAST...
    找到同源: sp|P12210.1| (identity: 100.0%, e-value: 2e-144)
    找到同源: sp|A0A360.1| (identity: 97.4%, e-value: 5e-142)
    找到同源: sp|Q9M3K5.1| (identity: 96.9%, e-value: 6e-141)
    找到同源: sp|Q09FT6.1| (identity: 98.0%, e-value: 1e-140)
    找到同源: sp|Q49KX3.1| (identity: 95.4%, e-value: 3e-140)
[2/100] 处理序列: sp|Q5B3K6|RNY1_EMENI (长度: 417)
  正在运行BLAST...
    找到同源: sp|Q5B3K6.1| (identity: 100.0%, e-value: 0e+00)
    找到同源: sp|Q4WXZ5.2| (identity: 65.6%, e-value: 0e+00)
    找到同源: sp|P24657.1| (identity: 55.6%, e-value: 2e-88)
    找到同源: sp|P10281.2| (identity: 50.2%, e-value: 2e-84)
    找到同源: sp|P19791.1| (identity: 49.8%, e-value: 4e-75)
[3/100] 处理序列: sp|Q6MM37|T23O_BDEBA (长度: 359)
  正在运行BLAST...
    找到同源: sp|Q6MM37.1| (identity: 100.0%, e-value: 0e+00)
    找到同源: sp|B4RUH2.1| (identity: 48.0%, e-value: 4e-125)
    找到同源: sp|Q55DB4.1| (identity: 44.5%, e-value: 4e-113)
    找到同源: sp|Q17P71.1| (identity: 45.7%, e-value: 

/root/miniconda3/lib/python3.12/site-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request N1VE8FMV016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


    找到同源: sp|Q7U564.1| (identity: 100.0%, e-value: 0e+00)
    找到同源: sp|Q3AWL5.1| (identity: 79.5%, e-value: 4e-154)
    找到同源: sp|Q7V652.1| (identity: 69.1%, e-value: 6e-138)
    找到同源: sp|A2C7F4.1| (identity: 68.4%, e-value: 3e-137)
    找到同源: sp|A2C0R7.1| (identity: 62.7%, e-value: 6e-125)
[10/100] 处理序列: sp|A9N941|ERA_COXBR (长度: 295)
  正在运行BLAST...


/root/miniconda3/lib/python3.12/site-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request N1WAWWS3016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


    找到同源: sp|A9N941.1| (identity: 100.0%, e-value: 0e+00)
    找到同源: sp|B6IZ00.1| (identity: 99.3%, e-value: 0e+00)
    找到同源: sp|A9KFA1.1| (identity: 99.0%, e-value: 0e+00)
    找到同源: sp|A5UFI7.2| (identity: 55.5%, e-value: 3e-113)
    找到同源: sp|Q9CPH8.1| (identity: 53.1%, e-value: 1e-110)
[11/100] 处理序列: sp|P41874|FAR3_PANRE (长度: 7)
  正在运行BLAST...


/root/miniconda3/lib/python3.12/site-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request N1XDCAVV016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


KeyboardInterrupt: 


KeyboardInterrupt

